# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* https://codingnomads.com/pytorch-dataset-to-dataloader-using-collate-fn
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from sklearn.datasets import make_regression
import torch as th

X,y = make_regression()
#X = th.from_numpy(X);
#y = th.from_numpy(y);

# можно th.tensor, другой класс th.Tensor
th.Tensor(X)
X = th.LongTensor(X)

display(X[0])
display(len(X))
print()

from torch.utils.data import Dataset
class RegressionDataset(Dataset):
  def __init__(self,transform = None):
    X,y = make_regression();
    self.transform = transform
    self.X = th.from_numpy(X); self.y = th.from_numpy(y);
    pass
  def __getitem__(self,idx):
    if self.transform is not None:
      return self.transform(self.X[idx]),self.y[idx];
    else:
      return self.X[idx],self.y[idx];
    pass
  def __len__(self):
    return len(self.X)
    pass

class MyTransform:
  def __init__(self,multiplier=None):
    self.multiplier = multiplier
  def __call__(self,x):
    return x*1000
dataset = RegressionDataset(transform=lambda x: x * 500)
print(dataset[0])

from torch.utils.data import DataLoader
loader = DataLoader(dataset,batch_size=16,shuffle=True,drop_last = True)
for X,y in loader:
  print(X.shape,y.shape)
  break

tensor([ 0,  0,  1,  0,  2,  1,  0, -1,  1,  1, -1,  0,  0,  0,  0,  0, -1,  0,
         1,  0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,
        -1,  0,  0,  0,  1,  0,  2,  0, -1,  0, -1,  0,  2, -1,  0,  0,  1,  0,
         0,  0,  0, -1,  0,  0,  0, -1,  0,  1,  0,  0, -1,  0, -1,  0,  0,  0,
         0,  1,  0,  0,  0,  2, -1,  0,  0,  0,  0,  1, -1,  0,  0,  0,  1,  0,
         0,  0,  0,  1,  0, -1,  0,  1,  2,  0])

100


(tensor([ -119.7769,   843.2219,    -6.1313,   985.7435,  -142.2576,  -541.6681,
         -963.8076,   -38.2008,  -449.2024,    14.1442,  -384.8599,   170.2980,
        -1189.3662,   138.4810,  -872.7808,   684.8506,   219.9532,   219.2017,
          303.9039,   234.3709,  -485.4854,   567.1229,  -181.0724, -1111.5179,
         -360.5583,  -488.5782,  -120.5171,   175.1987,   365.3563,    48.5316,
         -703.6241,   165.8763,  -744.8802,   -41.0799,   267.0469,   104.4217,
        -1264.2674,   121.1946,  -552.0803,   916.5870,   701.1308,    38.3971,
         -639.1084,    89.4149,   496.5798,   -64.2237,   -91.4803,  -335.2357,
          281.2204,  -445.7417,    47.9610,    76.9262,  -994.6764,  -481.0392,
            9.9411,  -376.2064,   152.4855,  -206.4686,    -7.1967,  -115.8623,
         -203.2232,  1102.2128,   607.7227,   184.9653,   346.3960,  -553.7171,
         -457.1677,  -974.9087,  -525.3688,   533.9993,  -379.0975,   337.4605,
           70.1568,  -469.3413,   168.

In [ ]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(X,y)
dataset

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
import torch as th
p = pd.read_csv('/content/bank-full.csv')
display(p.head())
class BankDatasetBase(
    th.utils.data.Dataset
):
    def __init__(self, data: pd.DataFrame, target) -> None:
        self.X = data.drop(target,axis=1)
        self.y = data[target]

    def __getitem__(self, idx: int):
        return (self.X.iloc[idx],self.y.iloc[idx]);

    def __len__(self) -> int:
        return len(self.X)

data = BankDatasetBase(p,'y')
display(data.X)
display(data[1])

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown


(age                  44
 job          technician
 marital          single
 education     secondary
 default              no
 balance              29
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            151
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 1, dtype: object,
 'no')

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [ ]:
from typing import Callable


class BankDataset(
    BankDatasetBase
):
    def __init__(
        self,
        data: pd.DataFrame,
        target,
        transform: Callable | None = None,
        target_transform: Callable | None = None
    ) -> None:
        super().__init__(data,target)
        self.transform = transform
        self.target_transform = target_transform;

    def __getitem__(self, idx: int) -> tuple:
        # x - набор признаков из idx-й строки
        # y - набор признаков из idx-й строки
        # если при создании был передан transform
        # x = transform(x)
        # если при создании был передан target_transform
        # y = target_transform(y)
        out = []
        if self.transform is not None: out.append(self.transform(self.X.iloc[idx]))
        else: out.append(self.X.iloc[idx])
        if self.target_transform is not None: out.append(self.target_transform(self.y.iloc[idx]))
        else: out.append(self.y.iloc[idx])
        return out


    def __len__(self) -> int:
        return len(self.X)

data = BankDataset(p,'y')
display(data.X)
data[1]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown


[age                  44
 job          technician
 marital          single
 education     secondary
 default              no
 balance              29
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            151
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 1, dtype: object,
 'no']

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию.

- [ ] Проверено на семинаре

In [ ]:
class OrdinalEncoderTransform():
  def __init__(self,names):
    self.names=names
    self.encoded = {name:{} for name in names}
    self.numbers = {name:0 for name in names}
  def __call__(self,features):
    k = features.copy()
    #print(type(k))
    for name in self.names:
      if k[name] not in self.encoded[name]:
        self.encoded[name].update({k[name]:self.numbers[name]});
        self.numbers[name] += 1
      k[name] = self.encoded[name][k[name]];
    return k
data = BankDataset(p,'y', transform = OrdinalEncoderTransform(p.drop('y',axis=1).select_dtypes(include=['object']).columns.tolist()))
data[0],data[1],data[2]

([age            58
  job             0
  marital         0
  education       0
  default         0
  balance      2143
  housing         0
  loan            0
  contact         0
  day             5
  month           0
  duration      261
  campaign        1
  pdays          -1
  previous        0
  poutcome        0
  Name: 0, dtype: object,
  'no'],
 [age           44
  job            1
  marital        1
  education      1
  default        0
  balance       29
  housing        0
  loan           0
  contact        0
  day            5
  month          0
  duration     151
  campaign       1
  pdays         -1
  previous       0
  poutcome       0
  Name: 1, dtype: object,
  'no'],
 [age          33
  job           2
  marital       0
  education     1
  default       0
  balance       2
  housing       0
  loan          1
  contact       0
  day           5
  month         0
  duration     76
  campaign      1
  pdays        -1
  previous      0
  poutcome      0
  Name: 2, dtype: 

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию.

- [ ] Проверено на семинаре

In [ ]:
class LabelEncoderTransform():
  def __init__(self):
    self.encoded = {}
    self.number = 0
  def __call__(self,label):
    if label not in self.encoded:
      self.encoded.update({label:self.number});
      self.number += 1
    return self.encoded[label]
data = BankDataset(p,'y', target_transform = LabelEncoderTransform())
data[1],data[2],data[3]


([age                  44
  job          technician
  marital          single
  education     secondary
  default              no
  balance              29
  housing             yes
  loan                 no
  contact         unknown
  day                   5
  month               may
  duration            151
  campaign              1
  pdays                -1
  previous              0
  poutcome        unknown
  Name: 1, dtype: object,
  0],
 [age                    33
  job          entrepreneur
  marital           married
  education       secondary
  default                no
  balance                 2
  housing               yes
  loan                  yes
  contact           unknown
  day                     5
  month                 may
  duration               76
  campaign                1
  pdays                  -1
  previous                0
  poutcome          unknown
  Name: 2, dtype: object,
  0],
 [age                   47
  job          blue-collar
  marital         

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию.

- [ ] Проверено на семинаре

In [73]:
class ToTensor():
  def __call__(self,x):
    return th.tensor(x.values)
import numpy as np
tr = ToTensor()
display(tr(pd.DataFrame(np.zeros((5,5)))))
display(tr(pd.Series(np.zeros(5))))


class LabelEncoderTransform():
  def __init__(self):
    self.encoded = {}
    self.number = 0
  def __call__(self,x,label):
    if label not in self.encoded:
      self.encoded.update({label:self.number});
      self.number += 1
    return (x,self.encoded[label])

class OrdinalEncoderTransform():
  def __init__(self,names):
    self.names=names
    self.encoded = {name:{} for name in names}
    self.numbers = {name:0 for name in names}
  def __call__(self,features,y):
    k = features.copy()
    for name in self.names:
      if k[name] not in self.encoded[name]:
        self.encoded[name].update({k[name]:self.numbers[name]});
        self.numbers[name] += 1
      k[name] = self.encoded[name][k[name]];
    return (k,y)

class ToTensor():
  def __call__(self,x,y):
    x = x.apply(np.float64)
    return (th.tensor(x.values),th.tensor(y))


class Compose():
  def __init__(self,transforms):
    self.transforms = transforms
  def __call__(self, x,y):
    for transform in self.transforms:
      x,y = transform(x,y)
    return x,y

class BankDataset(
    BankDatasetBase
):
    def __init__(
        self,
        data: pd.DataFrame,
        target,
        transform: Callable | None = None
    ) -> None:
        super().__init__(data,target)
        self.transform = transform

    def __getitem__(self, idx: int) -> tuple:
        if self.transform is not None:
          return self.transform(self.X.iloc[idx],self.y.iloc[idx])
        else:
          return self.X.iloc[idx],self.y.iloc[idx]


    def __len__(self) -> int:
        return len(self.X)

data = BankDataset(p,'y', transform = Compose([OrdinalEncoderTransform(p.drop('y',axis=1).select_dtypes(include=['object']).columns.tolist()),LabelEncoderTransform(),ToTensor()]))
data[0],data[1],data[2]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], dtype=torch.float64)

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

((tensor([ 5.8000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           2.1430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  2.6100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00], dtype=torch.float64),
  tensor(0)),
 (tensor([ 44.,   1.,   1.,   1.,   0.,  29.,   0.,   0.,   0.,   5.,   0., 151.,
            1.,  -1.,   0.,   0.], dtype=torch.float64),
  tensor(0)),
 (tensor([33.,  2.,  0.,  1.,  0.,  2.,  0.,  1.,  0.,  5.,  0., 76.,  1., -1.,
           0.,  0.], dtype=torch.float64),
  tensor(0)))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` (используйте аргумент `collate_fn`) таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.



- [ ] Проверено на семинаре

In [81]:
from torch.utils.data import DataLoader, random_split

cats = p.drop('y', axis=1).select_dtypes(include=['object']).columns.tolist()
dataset = BankDataset(p, 'y', transform=Compose([OrdinalEncoderTransform(cats),LabelEncoderTransform(),ToTensor()]))

train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

def custom_collate_fn(batch):
    features = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    features_batch = th.stack(features)
    labels_batch = th.stack(labels)

    features_ = features_batch.view(-1, 2, 8)
    return features_, labels_batch


train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle=True,collate_fn=custom_collate_fn)

batch_features, batch_labels = next(iter(train_dataloader))

print(batch_features.shape)
print(batch_labels.shape)

torch.Size([64, 2, 8])
torch.Size([64])
